In [18]:
!pip install kagglehub[pandas-datasets] pandas tqdm datasets transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/5e/0c/68d03a38f6ab2ba2b2829eb11b334610dd236e7926787f7656001b68e1f2/transformers-4.53.0-py3-none-any.whl.metadata
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/80/32/763a6cc01d21fb3819227a1cc3f60fd251c13c37c27a73b8ff4315433a8e/regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Obtaining dependency information for tokenizers<0.22,>=0.21 from https://files.pythonhosted.org/packages/13/c3/cc2755ee10be859c4338c962a35b9a663788c0c0b50c0bdd8078fb6870cf/tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata
  Obtaining dependency information for safetensors>=0.4.3 from https://files.pythonhosted.org/packages/69/e2/b011c38e5394c4c18fb5500778a55ec43ad6106126e74723ffaee246f56e/safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata
  Using cached safetensors-0.5.3-cp38-abi3-win_amd


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import ujson    # pip install ujson
import csv

# Paths
json_path = r'C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\raw\arxiv-metadata.json'
csv_path  = r'C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\processed\arxiv-metadata.csv'

# Fields to write
fields = [
    'id',
    'title',
    'abstract',
    'categories',
    'submitter',
    'doi',
    'journal-ref',
    'update_date',
    'first_version_date',
    'last_version_date',
    'authors',
    'authors_parsed'
]

with open(json_path, 'r', encoding='utf-8') as jf, \
     open(csv_path,  'w', newline='', encoding='utf-8') as cf:

    writer = csv.DictWriter(cf, fieldnames=fields)
    writer.writeheader()

    for i, line in enumerate(jf, 1):
        try:
            obj = ujson.loads(line)

            # Extract version dates
            versions = obj.get('versions', [])
            first_v = versions[0]['created'] if versions else ''
            last_v  = versions[-1]['created'] if versions else ''

            # Robust authors_parsed flattening
            parsed = []
            for name in obj.get('authors_parsed', []):
                if not isinstance(name, list) or len(name) == 0:
                    continue
                last = name[0]
                first = name[1] if len(name) > 1 else ''
                parsed.append(f"{last}, {first}")
            auth_parsed_str = '; '.join(parsed)

            row = {
                'id':                obj.get('id', ''),
                'title':             obj.get('title', '').replace('\n', ' ').strip(),
                'abstract':          obj.get('abstract', '').replace('\n', ' ').strip(),
                'categories':        obj.get('categories', ''),
                'submitter':         obj.get('submitter', ''),
                'doi':               obj.get('doi', ''),
                'journal-ref':       obj.get('journal-ref', ''),
                'update_date':       obj.get('update_date', ''),
                'first_version_date': first_v,
                'last_version_date':  last_v,
                'authors':           obj.get('authors', ''),
                'authors_parsed':    auth_parsed_str
            }
            writer.writerow(row)

        except Exception as e:
            print(f"Skipping line {i}: {e}")
            continue

        if i % 500000 == 0:
            print(f"Processed {i:,} lines...")

print("Done! CSV saved to:", csv_path)


Processed 500,000 lines...
Processed 1,000,000 lines...
Processed 1,500,000 lines...
Processed 2,000,000 lines...
Processed 2,500,000 lines...
Done! CSV saved to: C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\processed\arxiv-metadata.csv


In [3]:
import pandas as pd

csv_path  = r'C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\processed\arxiv-metadata.csv'
df = pd.read_csv(csv_path, encoding='utf-8')

C:\Users\nikhi\AppData\Local\Temp\ipykernel_14156\1713613826.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='utf-8')


In [7]:
df.head()

,id,title,abstract,categories,submitter,doi,journal-ref,update_date,first_version_date,last_version_date,authors,authors_parsed
0,704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturbati...,hep-ph,Pavel Nadolsky,10.1103/PhysRevD.76.013009,"Phys.Rev.D76:013009,2007",2008-11-26,"Mon, 2 Apr 2007 19:18:42 GMT","Tue, 24 Jul 2007 20:10:27 GMT","C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...","Balázs, C.; Berger, E. L.; Nadolsky, P. M.; Yu..."
1,704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-pe...",math.CO cs.CG,Louis Theran,NaN,NaN,2008-12-13,"Sat, 31 Mar 2007 02:26:18 GMT","Sat, 13 Dec 2008 17:26:00 GMT",Ileana Streinu and Louis Theran,"Streinu, Ileana; Theran, Louis"
2,704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is describe...,physics.gen-ph,Hongjun Pan,NaN,NaN,2008-01-13,"Sun, 1 Apr 2007 20:46:54 GMT","Sun, 13 Jan 2008 00:36:28 GMT",Hongjun Pan,"Pan, Hongjun"
3,704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle n...,math.CO,David Callan,NaN,NaN,2007-05-23,"Sat, 31 Mar 2007 03:16:14 GMT","Sat, 31 Mar 2007 03:16:14 GMT",David Callan,"Callan, David"
4,704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\Lam...,math.CA math.FA,Alberto Torchinsky,NaN,"Illinois J. Math. 52 (2008) no.2, 681-689",2013-10-15,"Mon, 2 Apr 2007 18:09:58 GMT","Mon, 2 Apr 2007 18:09:58 GMT",Wael Abu-Shammala and Alberto Torchinsky,"Abu-Shammala, Wael; Torchinsky, Alberto"


In [9]:
df.describe()

,id,title,abstract,categories,submitter,doi,journal-ref,update_date,first_version_date,last_version_date,authors,authors_parsed
count,2771128,2771128,2771128,2771128,2755938,1247563,897886,2771128,2771128,2771128,2771128,2771128
unique,2771104,2766890,2769305,90821,574133,1245291,886365,5417,2736601,2753251,2178104,2054357
top,math-ph/0512019,Discussion of: A statistical analysis of multi...,This paper has been withdrawn.,astro-ph,EPTCS,10.1145/1122445.1122456,"Dans Design, Automation and Test in Europe - D...",2007-05-23,"Wed, 27 Jun 2012 19:59:59 GMT","Wed, 27 Jun 2012 19:59:59 GMT",CMS Collaboration,"CMS Collaboration,"
freq,4,12,91,86911,4012,18,128,129945,109,103,1361,1361


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2771128 entries, 0 to 2771127
Data columns (total 12 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   id                  object
 1   title               object
 2   abstract            object
 3   categories          object
 4   submitter           object
 5   doi                 object
 6   journal-ref         object
 7   update_date         object
 8   first_version_date  object
 9   last_version_date   object
 10  authors             object
 11  authors_parsed      object
dtypes: object(12)
memory usage: 253.7+ MB


In [12]:
top20 = (
    df['categories']
    .value_counts()
    .head(20)
    .reset_index()
    .rename(columns={'index': 'category', 'categories': 'count'})
)

print(top20)

                count  count
0            astro-ph  86911
1              hep-ph  83510
2            quant-ph  74999
3               cs.CV  73704
4              hep-th  60534
5   cond-mat.mtrl-sci  42212
6             math.AP  37608
7   cond-mat.mes-hall  36528
8         astro-ph.GA  33880
9             math.CO  32460
10              gr-qc  32166
11              cs.CL  32008
12        astro-ph.SR  28152
13    cond-mat.str-el  27314
14            math.PR  25253
15        astro-ph.HE  25036
16      cs.IT math.IT  23794
17            math.NT  23786
18              cs.LG  23719
19        astro-ph.CO  23130


In [8]:
df.shape

(2771128, 12)

In [13]:
# Count null values in each column
null_counts = df.isnull().sum()

# Calculate percentage of null values
null_percentages = (null_counts / len(df)) * 100

# Combine counts and percentages in a summary DataFrame
null_summary = pd.DataFrame({
    'Null Count': null_counts,
    'Null Percentage': null_percentages
}).sort_values('Null Count', ascending=False)

print(null_summary)

                    Null Count  Null Percentage
journal-ref            1873242        67.598537
doi                    1523565        54.979958
submitter                15190         0.548152
id                           0         0.000000
categories                   0         0.000000
abstract                     0         0.000000
title                        0         0.000000
update_date                  0         0.000000
first_version_date           0         0.000000
last_version_date            0         0.000000
authors                      0         0.000000
authors_parsed               0         0.000000


In [4]:
df = df.drop(columns=['id', 'categories' ,'submitter', 'doi', 'journal-ref', 'update_date', 'first_version_date', 'last_version_date', 'authors', 'authors_parsed'])

In [5]:
df.head()

,title,abstract
0,Calculation of prompt diphoton production cros...,A fully differential calculation in perturbati...
1,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-pe..."
2,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is describe...
3,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle n...
4,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\Lam...


In [6]:
# save cleaned DataFrame to CSV
cleaned_csv_path = r'C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\processed\arxiv-metadata-cleaned.csv'
df.to_csv(cleaned_csv_path, index=False, encoding='utf-8')

In [2]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/f4/5a/ba91b8c57aa37dbd80d5ff958576a9a8c14317b04b671ae7f0d09b00993a/scikit_learn-1.7.0-cp311-cp311-win_amd64.whl.metadata
  Using cached scikit_learn-1.7.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Obtaining dependency information for scipy>=1.8.0 from https://files.pythonhosted.org/packages/8b/c9/750d34788288d64ffbc94fdb4562f40f609d3f5ef27ab4f3a4ad00c9033e/scipy-1.16.0-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.16.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/7d/4f/1195bbac8e0c2acc5f740661631d8d750dc38d4a32b23ee5df3cde6f4e0d/joblib-1.5.1-py3-none-any.whl.metadata
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/32/d5/f9a850d79b0851d1d4ef6456097579a9005b31fea687


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd

cleaned_csv_path = r'C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\processed\arxiv-metadata-cleaned.csv'

df = pd.read_csv(cleaned_csv_path, encoding='utf-8')

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_df.to_csv(r'C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\splits\arxiv-train.csv', index=False, encoding='utf-8')
val_df.to_csv(r'C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\splits\arxiv-val.csv', index=False, encoding='utf-8')
test_df.to_csv(r'C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\splits\arxiv-test.csv', index=False, encoding='utf-8')

In [1]:
from datasets import load_dataset


data_files = {
    "train": r"C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\splits\arxiv-train.csv",
    "validation": r"C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\splits\arxiv-val.csv",
    "test": r"C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\splits\arxiv-test.csv"
}
raw_datasets = load_dataset("csv", data_files=data_files)


c:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from datasets import load_dataset
import multiprocessing


def tokenize_function(examples):
    from transformers import PegasusTokenizer
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
    
    model_inputs = tokenizer(
        examples["abstract"],
        max_length=512,
        padding="max_length",
        truncation=True
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["title"],
            max_length=64,
            padding="max_length",
            truncation=True
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    num_proc=multiprocessing.cpu_count(),   
    load_from_cache_file=True               
)

tokenized_datasets.save_to_disk(r"C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\tokenized\arxiv-tokenized")


Saving the dataset (2/2 shards): 100%|██████████| 277113/277113 [00:05<00:00, 54283.43 examples/s]


In [8]:
# load the tokenized dataset
from datasets import load_from_disk
tokenized_datasets = load_from_disk(r"C:\Users\nikhi\OneDrive\Documents\GitHub\TitleForge\data\tokenized\arxiv-tokenized")

# Check the structure of the tokenized dataset
print(tokenized_datasets)
# Display the first few rows of the tokenized dataset
print(tokenized_datasets['train'].to_pandas().head())

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2216902
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 277113
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 277113
    })
})
                                           input_ids  \
0  [184, 5731, 109, 986, 113, 1273, 112, 9523, 14...   
1  [1825, 21924, 3792, 7002, 1625, 432, 27728, 25...   
2  [184, 692, 109, 15474, 37703, 575, 122, 45791,...   
3  [139, 2211, 1298, 640, 112, 5014, 7143, 108, 2...   
4  [139, 2560, 113, 109, 1972, 562, 117, 112, 120...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [1, 1, 1, 1, 1, 